In [1]:
import pandas as pd
from datetime import datetime
import os

In [5]:
# directory = "../dane/gtfs-drive/" to ma byc folder gdzie masz te foldery gtfs 
directory = "../data/gtfsy/"
day = '2023-06-15'

In [4]:
dates = ['2023-05-09', '2023-05-10', '2023-05-11', '2023-05-16',
         '2023-05-17', '2023-05-18', '2023-05-23', '2023-05-30',
         '2023-06-01', '2023-06-06', '2023-06-07', '2023-06-13',
         '2023-06-14', '2022-09-02', '2022-11-15', '2022-11-22']

In [41]:
def get_df(directory, day):
    filepath = directory + day
    extension = ".gtfs"
    routes_df = pd.read_csv(filepath + extension + "/routes.txt")
    stop_times_df = pd.read_csv(filepath + extension + "/stop_times.txt")
    trips_df = pd.read_csv(filepath + extension + "/trips.txt")
    rt_stop_times_df = pd.read_csv(filepath + '-rt.gtfs/stop_times.txt')
    merged_df = stop_times_df.merge(trips_df, on = 'trip_id').merge(routes_df, on = 'route_id').merge(rt_stop_times_df, on=['trip_id', 'stop_id'])
    merged_df['arrival_time_x'] = merged_df['arrival_time_x'].str.strip()
    merged_df['arrival_time_y'] = merged_df['arrival_time_y'].str.strip()
    

    # Create the past_midnight column

    merged_df['diff'] = pd.to_timedelta(merged_df['arrival_time_y'], errors='coerce') - pd.to_timedelta(merged_df['arrival_time_x'], errors='coerce')
    merged_df['diff'] = merged_df['diff'].apply(lambda x: pd.Timedelta.total_seconds(x))
    merged_df.reset_index(inplace=True)
    merged_df['id'] = merged_df.apply(lambda row: str(day).replace('-','') + str(row['index']).zfill(8), axis=1)   
    merged_df['date'] = day
    merged_df['r'] = day[0:4]
    merged_df['m'] = day[5:7]
    merged_df['d'] = day[8:10]
    merged_df = merged_df[["id","trip_id","route_short_name", "diff", "departure_time_x", "departure_time_y", "stop_id", "route_type", 'date', 'r', 'm', 'd']]
    merged_df.set_index("id", inplace=True)

    return merged_df

In [44]:
for d in dates:
    df = get_df(directory, d)
    df.to_csv("../data/exporty/"+str(d).replace('-','')+"_przejazdy.csv")